In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import string
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter
from sklearn import ensemble
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer, normalize
from sklearn.cluster import KMeans, MiniBatchKMeans

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('C://duh/Thinkful/Thinkful/Exercises/Medium_AggregatedData.csv')

In [3]:
df.head()

,audioVersionDurationSec,codeBlock,codeBlockCount,collectionId,createdDate,createdDatetime,firstPublishedDate,firstPublishedDatetime,imageCount,isSubscriptionLocked,...,slug,name,postCount,author,bio,userId,userName,usersFollowedByCount,usersFollowedCount,scrappedDate
0,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,blockchain,Blockchain,265164.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
1,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,samsung,Samsung,5708.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
2,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,...,it,It,3720.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
3,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,technology,Technology,166125.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104
4,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,...,robotics,Robotics,9103.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104


In [4]:
df = df[['text', 'title', 'author', 'language']]
df.drop_duplicates(subset='text', inplace=True)
df.drop(df.index[df.language != 'en'], inplace=True)
authors = df.author.value_counts()[:10].index.to_list()
df = df[df.author.isin(authors)]
df = df.reset_index(drop=True)

In [5]:
df.head()

,text,title,author,language
0,How Machine Learning is Revolutionizing Digita...,How Machine Learning is Revolutionizing Digita...,Yves Mulkers,en
1,"Tech & Telecom news — Mar 2, 2018\nPRODUCTS & ...","Tech & Telecom news — Mar 2, 2018",C Gavilanes,en
2,AI Biweekly: 10 Bits from May W3 — May W4\n\n\...,AI Biweekly: 10 Bits from May W3 — May W4,Synced,en
3,Pittsburgh’s Pivot to Artificial Intelligence\...,Pittsburgh’s Pivot to Artificial Intelligence,Synced,en
4,"Tech & Telecom news — Jun 14, 2018\nPRODUCTS &...","Tech & Telecom news — Jun 14, 2018",C Gavilanes,en


In [6]:
df.shape

(2024, 4)

In [7]:
nlp = spacy.load('en')
df['doc'] = None
df['bow'] = None

sentences = pd.DataFrame()

In [8]:
df.head()

,text,title,author,language,doc,bow
0,How Machine Learning is Revolutionizing Digita...,How Machine Learning is Revolutionizing Digita...,Yves Mulkers,en,None,None
1,"Tech & Telecom news — Mar 2, 2018\nPRODUCTS & ...","Tech & Telecom news — Mar 2, 2018",C Gavilanes,en,None,None
2,AI Biweekly: 10 Bits from May W3 — May W4\n\n\...,AI Biweekly: 10 Bits from May W3 — May W4,Synced,en,None,None
3,Pittsburgh’s Pivot to Artificial Intelligence\...,Pittsburgh’s Pivot to Artificial Intelligence,Synced,en,None,None
4,"Tech & Telecom news — Jun 14, 2018\nPRODUCTS &...","Tech & Telecom news — Jun 14, 2018",C Gavilanes,en,None,None


In [9]:
df.shape

(2024, 6)

In [10]:
for x in range(len(df.text)):
    if x % 250 == 0:
        print("Processing row {}".format(x))
    #df.text[x] = re.sub(df.title[x], '', df.text[x])
    df.text[x] = df.text[x].lower()
    df.text[x] = re.sub('https://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('http://*.*?\\n', ' ', df.text[x])
    df.text[x] = re.sub('\\n', ' ', df.text[x])
    df.text[x] = re.sub('bibliography.*', '', df.text[x])
    df.text[x] = re.sub('[%s]' % re.escape(string.punctuation), '', df.text[x])
    df.text[x] = re.sub('\w*\d\w*', '', df.text[x])
    df.text[x] = re.sub('’', '', df.text[x])
    df.text[x] = re.sub('—', ' ', df.text[x])
    df.text[x] = re.sub('”', '', df.text[x])
    df.text[x] = re.sub('“', '', df.text[x])
    df.text[x] = re.sub('‘', '', df.text[x])
    df.text[x] = re.sub('€', '', df.text[x])
    df.text[x] = re.sub('…', ' ', df.text[x])
    df.text[x] = re.sub('\ \ ', ' ', df.text[x])
    df.text[x] = re.sub('quest ce quune classe concrete google search févr la figure suivante représente nos classes classe animal nous avons bien notre classe mère animal et wwwgoogleie les décorateurs python est un langage de programmation qui nécessite décrire moins de lignes de code que le c ou le c il se veut openclassroomscom objet est une instance de classe google search en programmation orientée objet on appelle instance dune classe un objet avec un comportement et un état tous deux wwwgoogleie classe abstraite google search classe abstraite et interface quest ce quune classe abstraite une classe abstraite est une classe incomplète elle wwwgoogleie creer une classe python google search nov nous allons aussi essayer de comprendre les mécanismes de la programmation orientée objet en python wwwgoogleie première approche des classes python est un langage de programmation qui nécessite décrire moins de lignes de code que le c ou le c il se veut openclassroomscom', '', df.text[x])
    df.text[x] = df.text[x].strip()
    if df.text[x] == '':
        df.drop([x], inplace=True)
        df = df.reset_index(drop=True)

Processing row 0
Processing row 250
Processing row 500
Processing row 750
Processing row 1000
Processing row 1250
Processing row 1500
Processing row 1750
Processing row 2000


In [11]:
df.head()

,text,title,author,language,doc,bow
0,how machine learning is revolutionizing digita...,How Machine Learning is Revolutionizing Digita...,Yves Mulkers,en,None,None
1,tech telecom news mar products services vide...,"Tech & Telecom news — Mar 2, 2018",C Gavilanes,en,None,None
2,ai biweekly bits from may may may fujitsu a...,AI Biweekly: 10 Bits from May W3 — May W4,Synced,en,None,None
3,pittsburghs pivot to artificial intelligence s...,Pittsburgh’s Pivot to Artificial Intelligence,Synced,en,None,None
4,tech telecom news jun products services vide...,"Tech & Telecom news — Jun 14, 2018",C Gavilanes,en,None,None


In [12]:
df.shape

(2024, 6)

In [36]:
for x in range(len(df.text)):
    df.doc[x] = nlp(df.text[x])
    sents = [[sent, df.author[x]] for sent in df.doc[x]]
    sentences = sentences.append(sents, ignore_index=True)

In [37]:
sentences.head()

,0,1
0,"(how, machine, learning, is, revolutionizing, ...",Yves Mulkers
1,"(then, you, can, not, afford, to, wait, for, t...",Yves Mulkers
2,"(ai, and, machine, learning, ml, machines, tha...",Yves Mulkers
3,"(cognitive, functions, are, still, a, dream, o...",Yves Mulkers
4,"(ml, mimics, how, the, human, cognitive, syste...",Yves Mulkers


In [38]:
sentences.shape

(1108989, 2)

In [39]:
sentences

,0,1
0,"(how, machine, learning, is, revolutionizing, ...",Yves Mulkers
1,"(then, you, can, not, afford, to, wait, for, t...",Yves Mulkers
2,"(ai, and, machine, learning, ml, machines, tha...",Yves Mulkers
3,"(cognitive, functions, are, still, a, dream, o...",Yves Mulkers
4,"(ml, mimics, how, the, human, cognitive, syste...",Yves Mulkers
5,"(the, ml, data, analysis, is, based, on, the, ...",Yves Mulkers
6,"(they, now, beat, us, at, games, like, chess, ...",Yves Mulkers
7,"(they, can, recognize, images, more, accuratel...",Yves Mulkers
8,"(technology, let, us, look, at, some, examples...",Yves Mulkers
9,"(this, experience, might, be, taken, to, new, ...",Yves Mulkers


In [20]:
len(sentences[0][1])

131

In [21]:
def bag_of_words(text):
    
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    return [item[0] for item in Counter(allwords).most_common(25)]
    

def bow_features(sentences, common_words):
    
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    for i, sentence in enumerate(df['text_sentence']):
        
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        for word in words:
            df.loc[i, word] += 1
        
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [22]:
for x in range(len(df.text)):
    df.bow[x] = bag_of_words(df.doc[x])

In [23]:
df.head()

,text,title,author,language,doc,bow
0,how machine learning is revolutionizing digita...,How Machine Learning is Revolutionizing Digita...,Yves Mulkers,en,"(how, machine, learning, is, revolutionizing, ...","[ml, business, machine, technology, datum, ent..."
1,tech telecom news mar products services vide...,"Tech & Telecom news — Mar 2, 2018",C Gavilanes,en,"(tech, telecom, news, , mar, , products, ser...","[story, new, app, company, mobile, service, an..."
2,ai biweekly bits from may may may fujitsu a...,AI Biweekly: 10 Bits from May W3 — May W4,Synced,en,"(ai, biweekly, bits, from, may, , may, , may...","[ai, , intel, centre, develop, library, solut..."
3,pittsburghs pivot to artificial intelligence s...,Pittsburgh’s Pivot to Artificial Intelligence,Synced,en,"(pittsburghs, pivot, to, artificial, intellige...","[professor, robotic, robot, pittsburgh, year, ..."
4,tech telecom news jun products services vide...,"Tech & Telecom news — Jun 14, 2018",C Gavilanes,en,"(tech, telecom, news, , jun, , products, ser...","[story, app, new, announce, service, video, ga..."


In [33]:
common_words = set()
for x in range(len(df.text)):
    common_words.update(df.bow[x])

In [34]:
common_words

0       {human, technology, include, learning, machine...
1       {claim, revenue, disney,  , authentication, ne...
2       {million, develop, cancer, launch, technology,...
3       {company, pittsburgh, say, work, machine, cent...
4       {operator, time, week, tv, stock, build,  , ne...
5       {heatmap, lab, train, brain, sample, google, m...
6       {file, stackoverflowcom, create, powershell, a...
7       {tencent, amazon, human, unemployment, work, t...
8       {failure, human, production, rethink, chief, e...
9       {accord, crucial, lack, marketing, understandi...
10      {work, include, machine, learning, figure,  , ...
11      {far, baird, police, department, time, platfor...
12      {exist, lab, hellermark, time, spot, platform,...
13      {operator, cable, result, tmobile, sprint,  , ...
14      {sale, human, assistant, soon, think, google, ...
15      {doctor, dean, brain, patient, tool, informati...
16      {internet, concern, drive, growth, include,  ,...
17      {failu

In [29]:
word_counts = bow_features(sentences, common_words)

TypeError: unhashable type: 'set'

In [ ]:
word_counts.head()

In [ ]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0,
                                                    stratify=Y)

In [ ]:
#X_norm = normalize(X)

#pca = PCA(.95)
#X_pca = pca.fit_transform(X_norm)

svd= TruncatedSVD(10)
lsa = make_pipeline(svd, Normalizer(copy=False))
# Run SVD on the training data, then project the training data.
X_lsa = lsa.fit_transform(X)

# Calculate predicted values.
y_pred = KMeans(random_state=0).fit_predict(X_lsa)

In [ ]:
variance_explained=svd.explained_variance_ratio_
total_variance = variance_explained.sum()
print("Percent variance captured by all components:",total_variance*100)

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(X_lsa[:, 0], X_lsa[:, 1], c=word_counts.text_source.drop_duplicates())
plt.scatter(X_lsa[:, 2], X_lsa[:, 3], c=y_pred)
plt.scatter(X_lsa[:, 4], X_lsa[:, 5], c=y_pred)
plt.scatter(X_lsa[:, 6], X_lsa[:, 7], c=y_pred)
plt.scatter(X_lsa[:, 8], X_lsa[:, 9], c=y_pred)
plt.legend()
plt.show()

In [ ]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
        
print('Training score: ', lr.score(X_train, y_train))
print('Test score: ', lr.score(X_test, y_test))

In [ ]:
svm = SVC(kernel='linear')
train = svm.fit(X_train, y_train)

print('Training score: ', svm.score(X_train, y_train))
print('Test score: ', svm.score(X_test, y_test))